## 1. directly use APIs from module *openai*

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # load properties of .env into temperatory system environment variables
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
def get_completion(prompt, model_name="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    resp = openai.ChatCompletion.create(model=model_name, messages=messages, temperature=0)
    return resp.choices[0].message["content"]

In [4]:
get_completion("what is 1+1?")

'1+1 equals 2.'

## 2. create prompts by simple template (*Python string placeholder*)

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = "American English in a clam and respectful tone"

prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}. 
text: ```{customer_email}```
"""

get_completion(prompt)

## 3. use langchain's prompt template

### 3.1 create & use *ChatPromptTemplate*

In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
chat = ChatOpenAI(temperature=0)

In [15]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts import ChatPromptTemplate

# create ChatPromptTemplate object from template string
prompt_template = ChatPromptTemplate.from_template(template_string)

# check ChatPromptTemplate object
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [17]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_style = "American English in a clam and respectful tone"

In [29]:
# format messages by prompt template
customer_messages = prompt_template.format_messages(
    style=customer_style, 
    text=customer_email)

print(type(customer_messages))
print(len(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
1
<class 'langchain.schema.messages.HumanMessage'>


In [30]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American English in a clam and respectful tone. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [31]:
customer_response = chat(customer_messages)

print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


### 3.2 Reuse *ChatPromptTemplate* object

In [33]:
service_reply = """Key there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = "a polite tone that speaks in English Pirate"

In [34]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Key there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [36]:
serivce_response = chat(service_messages)

print(serivce_response.content)

Ahoy there, matey! I regret to inform ye that the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer mishandlin' yer blender by forgettin' to secure the lid afore settin' it in motion. Aye, tough luck, me heartie! Fare thee well!


## 4. Specific json output

### 4.1 by prompt template

Let's start with defining how we would like the LLM output to look like:

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [39]:
# parsing rules, output format and so on
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown. \

delivery_days: How many days did it take for the product to arrive? \
If this information is not found, output -1. \

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list. \

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [40]:
# create ChatPromptTemplate object from template string
review_prompot_template = ChatPromptTemplate.from_template(review_template)
print(review_prompot_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [47]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# format messages from prompot template
review_messages = review_prompot_template.format_messages(text=customer_review)
# print message
print(review_messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.




In [48]:
review_response = chat(review_message)

print(review_response.content)
print(type(review_response.content))

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
<class 'str'>


we got type str of output, how to get dict type output?

### 4.2 by output parser

In [51]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [54]:
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price,and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [55]:
# create output_parser from repsonse schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# create format instructions prompt by output_parser
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price,and output them as a comma separated Python list.
}
```


In [56]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown. \

delivery_days: How many days did it take for the product to arrive? \
If this information is not found, output -1. \

price_value: Extract any sentences about the value or price, \
and output them as a comma separated Python list. \

text: {text}

{format_instructions}
"""

In [57]:
review_prompt_template_2 = ChatPromptTemplate.from_template(template=review_template_2)

review_messages_2 = review_prompt_template_2.format_messages(
    text=customer_review, 
    format_instructions=format_instructions)

review_response_2 = chat(review_messages_2)
print(review_response_2.content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [66]:
review_result_2 = output_parser.parse(review_response_2.content)

print(review_result_2)
print(type(review_result_2))

{'gift': False, 'delivery_days': '2', 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}
<class 'dict'>


In [67]:
print(review_result_2.get('gift'))
print(review_result_2.get('delivery_days'))
print(review_result_2.get('price_value'))

False
2
["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
